# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

17 campaigns

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [130]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import time

In [131]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [132]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



All of the categorical features should be coerced into a numerical datatype. The rows that contain unknown values can also be dropped

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

The business objective of the task is to increase marketing campaign efficiency and effectiveness by identifying the main predictors of success.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [134]:
columns_to_encode = ['job', 'marital', 'default', 'housing', 'loan', 'contact']
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
encoded_features = onehot_encoder.fit_transform(df[columns_to_encode]).toarray()
feature_names = onehot_encoder.get_feature_names_out(columns_to_encode)
encoded_df = pd.DataFrame(encoded_features, columns=feature_names)

# Add the target feature
encoded_df = pd.concat([encoded_df, df['y']], axis=1)
label_encoder = LabelEncoder()
encoded_df['y'] = label_encoder.fit_transform(encoded_df['y'])
encoded_df.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,y
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [135]:
X = encoded_df
y = encoded_df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=41)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [136]:
#Baseline model
dummy_clf = DummyClassifier().fit(X_train, y_train)
baseline_score = dummy_clf.score(X_test, y_test)
print(baseline_score)

0.8896172210083354


We should aim to beat a score of ~0.89

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [137]:
start = time.time()
lgr = LogisticRegression().fit(X_train, y_train)
end = time.time()
lgr_time = end - start
lgr_train_score = lgr.score(X_train, y_train)
lgr_test_score = lgr.score(X_test, y_test)
print(lgr_train_score)
print(lgr_test_score)

1.0
1.0


### Problem 9: Score the Model

What is the accuracy of your model?

The model has 100% accuracy

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [138]:
start = time.time()
knn = KNeighborsClassifier().fit(X_train, y_train)
end = time.time()
knn_time = end - start
knn_train_score = knn.score(X_train.values, y_train)
knn_test_score = knn.score(X_test.values, y_test)
print(knn_train_score)
print(knn_test_score)

/Users/benjaminroth/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/Users/benjaminroth/anaconda3/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


0.9933751864312719
0.9907744598203447


In [139]:
start = time.time()
dtree = DecisionTreeClassifier().fit(X_train, y_train)
end = time.time()
dtree_time = end - start
dtree_train_score = dtree.score(X_train, y_train)
dtree_test_score = dtree.score(X_test, y_test)
print(dtree_train_score)
print(dtree_test_score)

1.0
1.0


In [140]:
start = time.time()
svc = SVC().fit(X_train, y_train)
end = time.time()
svc_time = end - start
svc_train_score = svc.score(X_train, y_train)
svc_test_score = svc.score(X_test, y_test)
print(svc_train_score)
print(svc_test_score)

1.0
1.0


In [141]:
models = [
    {'Model': 'Logistic Regression', 'Train Time': lgr_time, 'Train Accuracy': lgr_train_score, 'Test Accuracy': lgr_test_score},
    {'Model': 'KNN', 'Train Time': knn_time, 'Train Accuracy': knn_train_score, 'Test Accuracy': knn_test_score},
    {'Model': 'Decision Tree', 'Train Time': dtree_time, 'Train Accuracy': dtree_train_score, 'Test Accuracy': dtree_test_score},
    {'Model': 'SVM', 'Train Time': svc_time, 'Train Accuracy': svc_train_score, 'Test Accuracy': svc_test_score}
]

# Creating the DataFrame
models_df = pd.DataFrame(models)

models_df

,Model,Train Time,Train Accuracy,Test Accuracy
0,Logistic Regression,0.069094,1.000000,1.000000
1,KNN,0.006771,0.993375,0.990774
2,Decision Tree,0.012300,1.000000,1.000000
3,SVM,1.672949,1.000000,1.000000


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

Since the performance of all the models except KNN was 100% accuracy on both the test and training datasets, excessive hyperparameter tuning is probably unnecessary. However, I'll do some anyway to demonstrate

In [151]:
params = {'model__tol': [1e-5, 1e-4, 1e-3], 
          'model__C': [0.001, 0.1, 1]}
pipe = Pipeline([('scale', StandardScaler()), 
                 ('model', LogisticRegression())]).fit(X_train, y_train)

start = time.time()
grid = GridSearchCV(pipe, param_grid=params).fit(X_train, y_train)
end = time.time()

num_combinations = np.prod([len(v) for v in params.values()])
cv_folds = 5
# Calculate the total number of models
total_models = num_combinations * cv_folds
lgrTime = (end-start) / total_models

lgr_train_score = grid.score(X_train, y_train)
lgr_test_score = grid.score(X_test, y_test)
print(lgr_train_score)
print(lgr_test_score)
print(grid.best_params_)
print(lgrTime)

1.0
1.0
{'model__C': 0.001, 'model__tol': 1e-05}
0.04307562510172526


In [152]:
params = {'model__n_neighbors': [1, 5, 10, 100, 200, 500, 1000, 2000]}
pipe = Pipeline([('scale', StandardScaler()), 
                 ('model', KNeighborsClassifier())]).fit(X_train, y_train)

start = time.time()
grid = GridSearchCV(pipe, param_grid=params).fit(X_train, y_train)
end = time.time()

num_combinations = np.prod([len(v) for v in params.values()])
total_models = num_combinations * cv_folds
knnTime = (end-start) / total_models

knn_train_score = grid.score(X_train, y_train)
knn_test_score = grid.score(X_test, y_test)
print(knn_train_score)
print(knn_test_score)
print(grid.best_params_)
print(knnTime)

1.0
0.9995144452537024
{'model__n_neighbors': 1}
0.7994389533996582


In [153]:
params = {'criterion': ['gini', 'entropy', 'log_loss'], 
          'max_depth': [5, 10, 20, 50, 100, None],
          'min_samples_split': [2, 5, 10, 20, 50]}
model = DecisionTreeClassifier()

start = time.time()
grid = GridSearchCV(model, param_grid=params).fit(X_train, y_train)
end = time.time()
num_combinations = np.prod([len(v) for v in params.values()])
total_models = num_combinations * cv_folds
treeTime = (end-start) / total_models

tree_train_score = grid.score(X_train, y_train)
tree_test_score = grid.score(X_test, y_test)
print(tree_train_score)
print(tree_test_score)
print(grid.best_params_)
print(treeTime)

1.0
1.0
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2}
0.014223022990756565


In [154]:
params = {'model__kernel': ['rbf', 'poly', 'linear', 'sigmoid'],
         'model__gamma': [0.1, 1.0, 10.0, 100.0]}
pipe = Pipeline([('scale', StandardScaler()), 
                 ('model', SVC())]).fit(X_train, y_train)

start = time.time()
grid = GridSearchCV(pipe, param_grid=params).fit(X_train, y_train)
end = time.time()

num_combinations = np.prod([len(v) for v in params.values()])
total_models = num_combinations * cv_folds
svcTime = (end-start) / total_models

svc_train_score = grid.score(X_train, y_train)
svc_test_score = grid.score(X_test, y_test)
print(svc_train_score)
print(svc_test_score)
print(grid.best_params_)
print(svcTime)


1.0
1.0
{'model__gamma': 0.1, 'model__kernel': 'poly'}
1.28112253844738


All of the models had good performance in both the train and test datasets. However, I would pick the Decision Tree due to it having a lower training time.


## Business Objective
I would recommend that the advertising agency use a Decision Tree Classifier to identify which customers to target and which ones to avoid. This would help save money and time.

##### Questions